In [ ]:
# tmall_order_report.csv是订单数据，可挖掘的维度有订单时间、省份（收货地址），指标则
# 有销售量、销售额、退款金额、退货率、成交率、地区分布、下单时间趋势等。

In [1]:
# 数据理解及处理
import pandas as pd

data=pd.read_csv('tmall_order_report.csv')
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


In [2]:
data.info()  #数据集情况 ，共计28010条数据，6个字段

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


In [3]:
data.columns=data.columns.str.strip() # 去除掉列名的空格
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

In [4]:
data[data.duplicated()].count() # 没有完全重复的数据

订单编号        0
总金额         0
买家实际支付金额    0
收货地址        0
订单创建时间      0
订单付款时间      0
退款金额        0
dtype: int64

In [5]:
data.isnull().sum() # 付款时间存在空值，表示订单未支付

订单编号           0
总金额            0
买家实际支付金额       0
收货地址           0
订单创建时间         0
订单付款时间      3923
退款金额           0
dtype: int64

In [6]:
data['收货地址'].unique()

array(['上海', '内蒙古自治区', '安徽省', '湖南省', '江苏省', '浙江省', '天津', '北京', '四川省',
       '贵州省', '辽宁省', '河南省', '广西壮族自治区', '广东省', '福建省', '海南省', '江西省', '甘肃省',
       '河北省', '黑龙江省', '云南省', '重庆', '山西省', '吉林省', '山东省', '陕西省', '湖北省',
       '青海省', '新疆维吾尔自治区', '宁夏回族自治区', '西藏自治区'], dtype=object)

In [7]:
# 定义地区名称映射关系，将简化的地区名还原为标准名
area_mapping = {
    '上海': '上海市',
    '内蒙古': '内蒙古自治区',
    '安徽': '安徽省',
    '湖南': '湖南省',
    '江苏': '江苏省',
    '浙江': '浙江省',
    '天津': '天津市',
    '北京': '北京市',
    '四川': '四川省',
    '贵州': '贵州省',
    '辽宁': '辽宁省',
    '河南': '河南省',
    '广西': '广西壮族自治区',
    '广东': '广东省',
    '福建': '福建省',
    '海南': '海南省',
    '江西': '江西省',
    '甘肃': '甘肃省',
    '河北': '河北省',
    '黑龙江': '黑龙江省',
    '云南': '云南省',
    '重庆': '重庆市',
    '山西': '山西省',
    '吉林': '吉林省',
    '山东': '山东省',
    '陕西': '陕西省',
    '湖北': '湖北省',
    '青海': '青海省',
    '新疆': '新疆维吾尔自治区',
    '宁夏': '宁夏回族自治区',
    '西藏': '西藏自治区'
}
# 先对收货地址进行清洗，去除“自治区”“省”等字样（如果之前的清洗步骤保留了简称）
# 假设之前的清洗已经将“内蒙古自治区”处理为“内蒙古”等简称
data['收货地址'] = data['收货地址'].str.replace('自治区|维吾尔|回族|壮族|省', '')
# 使用映射字典转换为标准名称
data['收货地址'] = data['收货地址'].replace(area_mapping)
data['收货地址'].unique()

array(['上海市', '内蒙古自治区', '安徽省', '湖南省', '江苏省', '浙江省', '天津市', '北京市', '四川省',
       '贵州省', '辽宁省', '河南省', '广西壮族自治区', '广东省', '福建省', '海南省', '江西省', '甘肃省',
       '河北省', '黑龙江省', '云南省', '重庆市', '山西省', '吉林省', '山东省', '陕西省', '湖北省',
       '青海省', '新疆维吾尔自治区', '宁夏回族自治区', '西藏自治区'], dtype=object)

In [8]:
# 数据分析可视化

# 1、整体情况
result = {}
result['总订单数'] = data['订单编号'].count()
result['已完成订单数'] = data['订单编号'][data['订单付款时间'].notnull()].count()
result['未付款订单数'] = data['订单编号'][data['订单付款时间'].isnull()].count()
result['退款订单数'] = data['订单编号'][data['退款金额']>0].count()
result['总订单金额'] = data['总金额'][data['订单付款时间'].notnull()].sum()
result['总退款金额'] = data['退款金额'][data['订单付款时间'].notnull()].sum()
result['总实际收入金额'] = data['买家实际支付金额'][data['订单付款时间'].notnull()].sum()

In [9]:
result

{'总订单数': 28010,
 '已完成订单数': 24087,
 '未付款订单数': 3923,
 '退款订单数': 5646,
 '总订单金额': 2474823.0700000003,
 '总退款金额': 572335.9199999999,
 '总实际收入金额': 1902487.1500000001}

In [10]:
from pyecharts import options as opts
from pyecharts.charts import Map,Bar,Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker

table = Table()

headers = ['总订单数','总订单金额','已完成订单数','总实际收入金额','退款订单数','总退款金额','成交率','退货率']
rows = [
    [
        result['总订单数'],f"{result['总订单金额']/10000:.2f}万",
        result['已完成订单数'],f"{result['总实际收入金额']/10000:.2f}万",
        result['退款订单数'],f"{result['总订单金额']/10000:.2f}万",
        f"{result['已完成订单数']/result['总订单数']:.2%}",
        f"{result['退款订单数']/result['已完成订单数']:.2%}",
    ]
]
table.add(headers,rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='整体情况')
)
table.render_notebook()



总订单数,总订单金额,已完成订单数,总实际收入金额,退款订单数,总退款金额,成交率,退货率
28010,247.48万,24087,190.25万,5646,247.48万,85.99%,23.44%


In [13]:
# 2、地区分析
result2 = data[data['订单付款时间'].notnull()].groupby('收货地址').agg({'订单编号':'count'})
result21 = result2.to_dict()['订单编号']
print(result21)
c = (
    Map()
    .add("订单量",[*result21.items()],"china",is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='地区分布'),
        visualmap_opts=opts.VisualMapOpts(max_=1000),
    )
)

c.render("地区分布.html")

c.render_notebook()

{'上海市': 3060, '云南省': 667, '内蒙古自治区': 176, '北京市': 1853, '吉林省': 336, '四川省': 1752, '天津市': 1031, '宁夏回族自治区': 40, '安徽省': 528, '山东省': 1484, '山西省': 395, '广东省': 2022, '广西壮族自治区': 353, '新疆维吾尔自治区': 43, '江苏省': 1845, '江西省': 331, '河北省': 885, '河南省': 792, '浙江省': 1822, '海南省': 156, '湖北省': 57, '湖南省': 935, '甘肃省': 132, '福建省': 425, '西藏自治区': 2, '贵州省': 286, '辽宁省': 1012, '重庆市': 896, '陕西省': 441, '青海省': 18, '黑龙江省': 312}


In [14]:
# 时间分析

data['订单创建时间'] = pd.to_datetime(data['订单创建时间'])
data['订单付款时间'] = pd.to_datetime(data['订单付款时间'])

In [15]:
result31 = data.groupby(data['订单创建时间'].apply(lambda x:x.strftime("%Y-%m-%d"))).agg({'订单编号':'count'}).to_dict()['订单编号']
c = (
    Line()
    .add_xaxis(list(result31.keys()))
    .add_yaxis("订单量",list(result31.values()))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max",name="最大值"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="每日订单量走势"))
)
c.render("每日订单量走势.html")

'C:\\Users\\joh12n\\Desktop\\BigdataSpark\\OrderFromTmall\\每日订单量走势.html'

In [16]:
result32 = data.groupby(data['订单创建时间'].apply(lambda x:x.strftime("%H"))).agg({'订单编号':'count'}).to_dict()['订单编号']
x = [*result32.keys()]
y = [*result32.values()]
c = (
    Bar()
    .add_xaxis(x)
    .add_yaxis("订单量",y)
    .set_global_opts(title_opts=opts.TitleOpts(title="每小时订单量走势"))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max",name="峰值"),
                opts.MarkPointItem(name="第二峰值",coord=[x[15],y[15]],value=y[15]),
                opts.MarkPointItem(name="第三峰值",coord=[x[10],y[10]],value=y[10]),
            ]
        ),
    )
    
)
c.render("每小时订单量走势.html")

'C:\\Users\\joh12n\\Desktop\\BigdataSpark\\OrderFromTmall\\每小时订单量走势.html'

In [17]:
s = data['订单付款时间'] - data['订单创建时间']
s[s.notnull()].apply(lambda x:x.seconds / 60).mean()  # 从下单到付款平均耗时7.7分钟

7.7399046511949745